In [1]:
from model_wrapper import NB101Wrapper,NB201Wrapper,NBNLPWrapper,DARTSWrapper,ResNetSeriesWrapper

In [2]:
import json

In [3]:
import torch

In [4]:
import time

In [5]:
from fbnas import NDS
from pycls.models.nas.genotypes import GENOTYPES, Genotype
import os

In [6]:
from builder_model import ModelBuilder

In [7]:
search_space = NDS('DARTS','../../GenNAS/data/nds_data/')

In [8]:
from argparse import Namespace
from builder_evaluator import Evaluator
from builder_task import CVTask,NLPTask
from configs import *

In [9]:
args = {'device':'cuda',\
        'config':{'last_channel_l0':0.25,'last_channel_l1':0.50,'last_channel_l2':0.75,'dimension': 64,'batch_size': 64},\
       'output_size': 8,
       'nds_path': '../../GenNAS/data/nds_data/',
       'search_space': 'DARTS',
       'last_channels': 64,
       'init_channels': None,
        
       }
args = Namespace(**args) 

In [10]:
args.total_iters = 100
args.eval_interval = 10
args.init_w_type = 'none'
args.init_b_type = 'none'
args.learning_rate = 1e-1
args.weight_decay = 4e-5
args.momentum = 0.9
args.train_weights = [1,1,1]
args.eval_weights = [1,1,1]
args.mutate_ratio = 1.
args.neighbor = 200
args.nlp_path = 'data/nasbenchnlp'
args.config = CONF_NB101
args.dataset = 'cifar10'
args.workers = 2
args.pad = False
args.input_size = 32
args.output_size = 8
args.length = 8
args.batch_size = None
cv_task = CVTask(args)

Files already downloaded and verified
Files already downloaded and verified


In [11]:
evaluator = Evaluator(args)

In [12]:
for search_space in ['DARTS','DARTS_in','DARTS_fix-w-d','DARTS_fix-w-d_in',\
                                     'ENAS','ENAS_in','ENAS_fix-w-d',\
                                     'PNAS','PNAS_in','PNAS_fix-w-d',\
                                     'Amoeba','Amoeba_in',\
                                     'NASNet','NASNet_in','ResNet','ResNeXt-A','ResNeXt-A_in','ResNeXt-B','ResNeXt-B_in']:
    args.search_space = search_space
    model_builder = ModelBuilder(args)
    s0 = time.time()
    evaluator.evaluate(cv_task, model_builder, 0)
    print(time.time() - s0)

10.443368196487427
7.6586456298828125
6.7892725467681885
18.838331699371338
16.79702615737915
8.767149686813354
7.266899347305298
8.417202711105347
4.152502775192261
9.759689569473267
9.419320344924927
7.913186311721802
7.964836359024048
4.50214147567749
2.9317305088043213
3.207308053970337
1.264204740524292
1.1731061935424805
3.3249855041503906


In [13]:
args.search_space = 'nasbench101'
args.init_channels = 16
model_builder = ModelBuilder(args)

arch = [[[0, 1, 0, 0, 0, 0, 0],
  [0, 0, 1, 0, 0, 1, 0],
  [0, 0, 0, 1, 1, 1, 0],
  [0, 0, 0, 0, 0, 1, 0],
  [0, 0, 0, 0, 0, 1, 0],
  [0, 0, 0, 0, 0, 0, 1],
  [0, 0, 0, 0, 0, 0, 0]],
 ['input',
  'conv1x1-bn-relu',
  'conv1x1-bn-relu',
  'conv3x3-bn-relu',
  'conv1x1-bn-relu',
  'maxpool3x3',
  'output']]
s0 = time.time()
evaluator.evaluate(cv_task, model_builder, arch)
print(time.time() - s0)

2.056891441345215


In [14]:
args.search_space = 'nasbench201'
args.init_channels = 16
model_builder = ModelBuilder(args)
arch = [[0],[0,1],[0,1,2]]
s0 = time.time()
evaluator.evaluate(cv_task, model_builder, arch)
print(time.time() - s0)

1.5722935199737549


In [26]:
args.search_space = 'nasbenchnlp'
args.last_channels = 16
args.config = CONF_NLP
nlp_task = NLPTask(args)
evaluator = Evaluator(args)
all_stats = []
for fn in os.listdir('./data/nasbenchnlp/train_logs_single_run/'):
    if fn.endswith('.json'):
        all_stats.append(json.load(open(os.path.join('./data/nasbenchnlp/train_logs_single_run', fn), 'r')))
ok_stats = [x for x in all_stats if x['status'] == 'OK']
sampled_stat = []
for indx,stat in enumerate(ok_stats):
    sampled_stat.append([indx,stat])

model_builder = ModelBuilder(args)

In [28]:
explorer = Explorer(model_builder,args)

In [42]:
spec = explorer.random_spec()
print(spec[1]['recepie'])
explorer.mutate_spec(spec)[1]['recepie']

{"node_0": {"op": "linear", "input": ["h_prev_2", "h_prev_0"]}, "node_1": {"op": "activation_sigm", "input": ["node_0"]}, "node_2": {"op": "linear", "input": ["node_1", "h_prev_1"]}, "node_3": {"op": "activation_leaky_relu", "input": ["node_2"]}, "node_4": {"op": "linear", "input": ["x", "node_3"]}, "node_5": {"op": "activation_leaky_relu", "input": ["node_4"]}, "node_9": {"op": "linear", "input": ["h_prev_0", "node_3", "h_prev_2"]}, "h_new_0": {"op": "blend", "input": ["h_new_2", "node_5", "h_new_1"]}, "h_new_1": {"op": "activation_tanh", "input": ["node_9"]}, "h_new_2": {"op": "linear", "input": ["h_new_1", "node_1"]}}


'{"node_0": {"op": "elementwise_prod", "input": ["h_prev_1", "h_prev_0"]}, "node_2": {"op": "linear", "input": ["x", "h_prev_1"]}, "node_3": {"op": "activation_leaky_relu", "input": ["node_2"]}, "node_4": {"op": "linear", "input": ["h_prev_0", "node_3"]}, "node_6": {"op": "elementwise_sum", "input": ["node_4", "node_2"]}, "node_7": {"op": "blend", "input": ["node_4", "h_prev_0", "node_0"]}, "node_8": {"op": "blend", "input": ["node_2", "node_7", "node_6"]}, "node_10": {"op": "linear", "input": ["h_prev_1", "x"]}, "h_new_0": {"op": "blend", "input": ["node_8", "h_new_1", "node_3"]}, "h_new_1": {"op": "activation_leaky_relu", "input": ["node_10"]}}'

In [16]:
s0 = time.time()
evaluator.evaluate(nlp_task, model_builder, [0,sampled_stat[0]])
print(time.time() - s0)

2.2799882888793945


In [17]:
from builder_explorer import Explorer

In [18]:
args.search_space = 'nasbench101'
args.config = CONF_NB101
model_builder = ModelBuilder(args)

In [19]:
explorer = Explorer(model_builder,args)

In [20]:
spec = explorer.random_spec()
explorer.mutate_spec(spec)

[array([[0, 0, 0, 1, 0, 1, 0],
        [0, 0, 0, 0, 0, 1, 0],
        [0, 0, 0, 0, 0, 0, 1],
        [0, 0, 0, 0, 0, 1, 0],
        [0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 1],
        [0, 0, 0, 0, 0, 0, 0]]),
 ['input',
  'conv1x1-bn-relu',
  'conv3x3-bn-relu',
  'conv1x1-bn-relu',
  'conv1x1-bn-relu',
  'maxpool3x3',
  'output']]

In [44]:
with open('./test.json','w') as t:
    json.dump(explorer.mutate_spec(spec)[1],t)